In [385]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import mysql.connector 

In [386]:
sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1:3307/consultation')
dbConnection    = sqlEngine.connect()

In [387]:
sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1:3307/client')
dbConnection1    = sqlEngine.connect()

In [388]:
sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1:3307/facility')
dbConnection2   = sqlEngine.connect()

In [389]:
data = pd.read_sql("select art_number,person_id,visit_type,tb_status,state from art a join art_current_status c on a.art_id = c.art_id join art_visit d on a.art_id=d.art_id", dbConnection)

In [390]:
med = pd.read_sql("SELECT person_id, date, medicine FROM consultation.person_medication", dbConnection)

In [391]:
med.head()

,person_id,date,medicine
0,c2453634-9dd4-4990-a0ad-a52b27ef2087,2012-04-12,TENOLAM E(TENOFOVIR DISOPROXIL LAMIVUDINE AND ...
1,53061d09-0ea9-43b6-ba37-4df9b5e12f14,2021-07-13,RIFAMPICIN PYRAZINAMIDE ETHAMBUTOL AND ISONIAZID
2,8443cb42-86c4-4520-b014-1d69e546d4ec,2020-12-14,CEFTRIAXONE
3,c2453634-9dd4-4990-a0ad-a52b27ef2087,2021-06-09,CEFTRIAXONE
4,1511df15-4054-4dd9-b89c-aa37c5d5cbca,2021-07-16,CLOFAZIMINE


In [392]:
ArtID = pd.read_sql("select art_id,person_id from art",dbConnection)

In [393]:
dataWithArtID = data.merge(ArtID, on="person_id")

In [394]:
dataWithArtID.head()

,art_number,person_id,visit_type,tb_status,state,art_id
0,09-OA-27-2014-A-00342,41585661-cf06-48f8-9e65-582bfa32b99f,None,None,CONTINUE,2c9384af-8eed-4e90-bf00-331b07060461
1,07-06-28-2020-A-00080,8443cb42-86c4-4520-b014-1d69e546d4ec,None,None,CONTINUE,1ad32d95-4644-4ef5-bd08-c12472e96d4a


In [395]:
visitDate = pd.read_sql("select art_id,state,date from art_current_status",dbConnection)

In [396]:
DatedDatas = pd.merge(dataWithArtID,visitDate, how='left',left_on=['art_id','state'],right_on=['art_id','state'])

In [397]:
DatedData  = pd.merge(DatedDatas,med, how='left',left_on=['person_id','date'],right_on=['person_id','date'])

In [398]:
DatedData.head()

,art_number,person_id,visit_type,tb_status,state,art_id,date,medicine
0,09-OA-27-2014-A-00342,41585661-cf06-48f8-9e65-582bfa32b99f,None,None,CONTINUE,2c9384af-8eed-4e90-bf00-331b07060461,2020-06-30,NaN
1,07-06-28-2020-A-00080,8443cb42-86c4-4520-b014-1d69e546d4ec,None,None,CONTINUE,1ad32d95-4644-4ef5-bd08-c12472e96d4a,2020-10-06,NaN


In [399]:
client = pd.read_sql("select person_id,firstname,lastname,birthdate,sex from person", dbConnection1);

In [400]:
merged = DatedData.merge(client, on="person_id")

In [401]:
trimmedArtNumbers = data['art_number'].str[:8]

In [402]:
out = trimmedArtNumbers.value_counts().index

In [403]:
s = out[0].replace('-','') #replace s with w then on 90 replace w with s

In [404]:
w = "ZW000A42"  #when site name is not found force with the name of the database ie replace w with s

In [405]:
facilityConfig = pd.read_sql("select * from facility a join district b on a.district_id = b.district_id join province c on a.province_id = c.province_id",dbConnection2)

In [406]:
facilityConfig

,facility_id,district_id,name,province_id,dhis_code,longitude,latitude,district_id,name,province_id,province_id,name
0,ZW000A01,ZW000A,Avondale Clinic,ZW00,LixFsP9YMT9,31.0291,-17.7883,ZW000A,Harare District,ZW00,ZW00,Harare Province
1,ZW000A02,ZW000A,Arcadia Primary Care Clinic,ZW00,OBXlKhRkRIb,31.0546,-17.8468,ZW000A,Harare District,ZW00,ZW00,Harare Province
2,ZW000A05,ZW000A,Belvedere FHS,ZW00,UiPBTlrwap5,31.0243,-17.8200,ZW000A,Harare District,ZW00,ZW00,Harare Province
3,ZW000A07,ZW000A,Borrowdale Primary Care Clinic,ZW00,WaYcDd2d9pk,0.0000,0.0000,ZW000A,Harare District,ZW00,ZW00,Harare Province
4,ZW000A08,ZW000A,Braeside FHS,ZW00,t0hAcwHFHPq,31.0657,-17.8399,ZW000A,Harare District,ZW00,ZW00,Harare Province
...,...,...,...,...,...,...,...,...,...,...,...,...
1523,ZW090A27,ZW090A,Cowdray Park,ZW09,bCj2JfVpDwN,28.5087,-20.0798,ZW090A,Bulawayo District,ZW09,ZW09,Bulawayo Province
1524,ZW090A31,ZW090A,24-Hr Medical centre,ZW09,VWMkdfPIFPU,28.5277,-20.1959,ZW090A,Bulawayo District,ZW09,ZW09,Bulawayo Province
1525,ZW090A33,ZW090A,Maqhawe,ZW09,RPASqynqwCM,28.5009,-20.1825,ZW090A,Bulawayo District,ZW09,ZW09,Bulawayo Province
1526,ZW090A35,ZW090A,Pumula South,ZW09,Ta5WU3NkNI0,28.4789,-20.1622,ZW090A,Bulawayo District,ZW09,ZW09,Bulawayo Province


In [407]:
facilityConfig.drop(['district_id','province_id','dhis_code','longitude','latitude'], axis=1,inplace=True)

In [408]:
facilityConf = facilityConfig[facilityConfig['facility_id'].str.contains(s)]

In [409]:
province = facilityConf['facility_id'].str[2:4]

In [410]:
district = facilityConf['facility_id'].str[4:6]

In [411]:
faci = facilityConf['facility_id'].str[6:]

In [412]:
facilityConf['FacilityCode'] = faci

In [413]:
facilityConf['DistrictCode'] = district

In [414]:
facilityConf['ProvinceCode'] = province

In [415]:
facilityConf.columns.values[1] = "FacilityName"

In [416]:
facilityConf.columns.values[2] = "DistrictName"

In [417]:
facilityConf.columns.values[3] = "ProvinceName"

In [418]:
facilityConf

,facility_id,FacilityName,DistrictName,ProvinceName,FacilityCode,DistrictCode,ProvinceCode


In [419]:
facility = facilityConf

In [420]:
a = facility['FacilityName'].str[:45].value_counts().index
b = a[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
c = facility['FacilityCode'].str[:20].value_counts().index
d = c[0]

In [ ]:
e = facility['DistrictCode'].str[:20].value_counts().index
f = e[0]

In [ ]:
g = facility['ProvinceCode'].str[:20].value_counts().index
h = g[0]

In [ ]:
i = facility['DistrictName'].str[:45].value_counts().index
j = i[0]

In [ ]:
k = facility['ProvinceName'].str[:45].value_counts().index
l = k[0]

In [ ]:
merged.drop(['person_id','art_id'], axis=1,inplace=True)

In [ ]:
clearMerged = merged.assign(FacilityName=b,DistrictName=j,ProvinceName=l,FacilityCode=d,DistrictCode=f,ProvinceCode=h)

In [ ]:
clearMerged.columns.values[3] = "art_status"

In [ ]:
clearMerged.columns.values[4] = "visit_date"

In [ ]:
clearMerged.head()

In [ ]:
clearMerged.to_csv('/home/tinashe/Documents/tbscreeningQ2/%s.csv'%b, encoding='utf-8', index=False)

In [ ]:
dbConnection.close()
dbConnection1.close()
dbConnection2.close()